# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env") 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [2]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPStatusError as e:
            # Handle 302 redirect manually if follow_redirects fails
            if e.response.status_code == 302 and "location" in e.response.headers:
                redirect_url = e.response.headers["location"]
                if not redirect_url.startswith("http"):
                    # handle relative redirects
                    redirect_url = urljoin(url, redirect_url)
                try:
                    response = await client.get(redirect_url, headers=headers)
                    response.raise_for_status()
                except Exception as e2:
                    print(f"Redirect request failed: {e2}")
                    return ""
            else:
                print(f"Request failed: {e}")
                return ""
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [3]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]
    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 인덕션 추천
LLM Query: 삼성전자 인덕션 제품 중 2구 모델이 아닌 3구 이상 또는 다른 화구 수를 가진 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 알려주세요.
Google Search Results: 5


삼성전자 인덕션 제품 중 3구 이상 또는 다양한 화구 구성을 가진 모델 중에서 특히 추천할 만한 제품들을 소개해 드립니다. 최신 2025년 5월 기준으로 가성비, 성능, 편의성 모두 뛰어난 모델 위주로 안내해 드립니다.

---

## 1. 삼성전자 비스포크 AI 인덕션 3구 NZ63DB607CF (2024년형)

### 주요 기능 및 특징
- **출력**: 대화구 3,400W, 중화구 2,600W, 소화구 1,200W로 상위 10% 최상급 화력
- **편의 기능**: AI 넘침 방지 모드, 저소음 기능(DCN), 대화형 알림창 탑재
- **디자인**: 프레임리스 마감으로 심플하고 얇은 두께, 식기세척기와 설치 시 튀어나오는 부분 최소화
- **조작부**: 1개의 심플한 조작부, 직관적인 사용 가능
- **가격**: 동급 화력 대비 100만 원 미만으로 가성비 우수

### 장점
- 초고화력으로 빠른 조리 가능
- AI 기반 편의 기능으로 사용 편리성 향상
- 세련된 디자인과 얇은 두께로 설치 및 인테리어 적합
- 합리적인 가격으로 삼성 비스포크 라인에서 인기

---

## 2. 삼성전자 비스포크 인덕션 3구 NZ63D650BXE (하위 모델)

### 주요 기능 및 특징
- **출력**: NZ63DB607CF와 동급 (대화구 3,400W 등)
- **편의 기능**: AI 끓음 감지 기능 미탑재, 각 화구별 조작부 3개
- **두께**: 상기 모델 대비 다소 두꺼움
- **가격**: 66~79만원대, 상대적으로 저렴

### 장점
- 뛰어난 출력과 기본 기능 충실
- 예산을 절약하고 싶을 때 좋은 선택
- 삼성 비스포크 디자인 적용

---

## 3. 삼성전자 비스포크 AI 인덕션 플렉스존 3구 NZ63DB657CFH (프리미엄 모델)

### 주요 기능 및 특징
- **출력**: 각 화구 일반 출력 2,000W 이상으로 강력
- **플렉스존**: 4분할 플렉스존으로 다양한 크기의 조리기구 자유롭게 배치 가능
- **편의 기능**: LCD 대화형 알림창, AI 끓음 감지, 고주파 소음 저감 DCN 기능, 앱 연동 원격 제어
- **디자인**: 얇은 두께, 프레임리스 마감으로 모던하고 깔끔함
- **가격**: 프리미엄급이지만 합리적인 가격대 유지

### 장점
- 다용도 플렉스존으로 유연한 조리 공간 활용 가능
- 스마트 기능과 안전 기능 강화로 고급 사용자 만족도 높음
- 프리미엄 인덕션 중 가성비 우수

---

## 4. 삼성전자 더 플레이트 인덕션 2구 NZ62DG300CFW (참고용)

- 2구 모델이나, 3구 이상의 인덕션과 차별화된 세련된 디자인과 마감, 뛰어난 성능으로 2구 인덕션 대표 모델임

---

## 요약 및 추천

| 모델명              | 화구수 | 최대 출력 (대화구 기준) | 주요 기능                        | 가격대(원)  | 추천 포인트                     |
|---------------------|--------|-------------------------|--------------------------------|-------------|-------------------------------|
| NZ63DB607CF         | 3구    | 3,400W                  | AI 넘침 방지, 저소음, 대화형 알림창 | 90~100만 원 | 최고 화력, 가성비, 디자인     |
| NZ63D650BXE         | 3구    | 3,400W                  | 기본 AI 기능 제외, 각 화구별 조작부 3개 | 66~79만 원  | 가성비 좋은 기본형            |
| NZ63DB657CFH        | 3구    | 2,000W 이상             | 4분할 플렉스존, AI 끓음 감지, 앱 원격제어 | 110~130만 원| 프리미엄 기능, 플렉스존 탑재  |

---

### 결론
- **최고의 가성비와 화력을 원한다면**: 삼성 비스포크 AI 인덕션 NZ63DB607CF 추천
- **예산 절감이 중요하다면**: NZ63D650BXE 모델이 적합
- **프리미엄 기능과 스마트 편의성을 원한다면**: NZ63DB657CFH 모델이 최선

삼성전자 비스포크 인덕션 라인은 뛰어난 화력과 AI 기반 편의 기능, 세련된 디자인을 고루 갖추고 있어 주방 공간과 요리 스타일에 맞춰 선택하시면 좋은 만족도를 얻으실 수 있습니다.

필요시 추가 정보나 설치 관련 상담도 도와드리니 편하게 문의해주세요!

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님 선물용으로 적합한 삼성전자 TV 모델을 추천해 주세요. 사용 용도와 예산에 맞는 삼성전자 TV의 주요 기능과 장점을 알려 주세요.
Google Search Results: 5
Request failed: 


안녕하세요! 2025년 5월 현재, 부모님 선물용 삼성전자 TV를 추천해 드리겠습니다.  
아래는 약 130만원 예산에 맞고, 부모님께서 주로 자연 다큐나 여행기 시청 등 화질이 중요한 용도에 적합한 삼성전자 TV 모델과 주요 기능, 장점입니다.

---

## 1. 추천 삼성전자 TV 모델

### 삼성 QLED 60~70인치 라인업 (예: 삼성 Q60B, Q67B, Q70B 등)
- 75인치 모델도 있으나 130만원 예산에서 70인치대 QLED 모델이 현실적인 선택입니다.
- QLED 기술로 선명하고 밝은 화질 제공
- 최신 스마트 TV 기능 탑재

---

## 2. 주요 기능과 장점

| 기능 | 설명 | 부모님 사용에 적합한 이유 |
|---|---|---|
| **QLED 디스플레이** | 퀀텀닷 기술로 색 재현율과 밝기 우수 | 자연 다큐, 여행기 영상에서 생생한 색감과 밝기 경험 가능 |
| **4K 해상도** | 고해상도 영상 지원 | 선명한 화질로 작은 디테일까지 표현 |
| **스마트 TV 기능** | 유튜브, 넷플릭스 등 다양한 앱 실행 가능 | 편리하게 다양한 콘텐츠 감상 가능 |
| **음성 인식 기능 (빅스비 등)** | 음성으로 채널 변경, 검색 가능 | 리모컨 조작이 어려운 부모님께 편리함 제공 |
| **매직스크린(탭뷰) 연동** | 스마트폰 화면 TV로 간편 전송 | 사진, 동영상 쉽게 공유 가능 |
| **삼성전자 A/S 및 브랜드 신뢰성** | 국내 A/S 네트워크 우수 | 빠르고 신뢰할 수 있는 사후 지원 보장 |

---

## 3. 참고 사항

- TCL 등 타 브랜드 QLED 모델도 있으나, 부모님 세대는 삼성/엘지 등 국내 대기업 브랜드에 대한 선호도가 높고, A/S 측면에서도 안정적입니다.
- 보급형 모델도 있으나, 130만원 예산에서는 Q60B~Q70B급 QLED 모델을 선택하는 것이 화질과 사용 편의성에서 좋은 균형을 이룹니다.
- 연말 할인 행사나 특가 이벤트를 활용하면 75인치 이상 모델도 예산에 맞출 수 있으니, 쿠팡 등 온라인몰 핫딜을 주시하는 것도 추천드립니다.

---

## 요약

| 추천 모델 | 삼성 Q60B, Q67B, Q70B (60~70인치 QLED) |
|---|---|
| 가격대 | 약 120~130만원대 (할인 시 변동 가능) |
| 주요 장점 | 뛰어난 화질, 스마트 기능, 국내 A/S 편리성 |

부모님께 선물용으로 적합하며, 화질과 사용 편의성을 모두 갖춘 모델입니다. 추가로 필요하신 정보가 있으면 언제든 문의 주세요!

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 개선점 작년 대비
LLM Query: 삼성전자 2025년 제품이 작년 제품과 비교해 어떤 점에서 개선되고 좋아졌는지 구체적인 변화와 기능 향상 부분을 알려주세요.
Google Search Results: 5


...

삼성전자에서 판매하는 **JBL**과 **하만카돈(Harman/Kardon)** 스피커는 모두 하만 인터내셔널(Harman International) 산하 브랜드로, 각기 다른 특징과 장점을 가지고 있습니다. 두 브랜드는 음질, 디자인, 그리고 기술적 차별점에서 명확한 차이가 있으니, 2025년 5월 기준 최신 정보를 바탕으로 자세히 비교해 드리겠습니다.

---

## 1. 브랜드 개요

- **JBL**
  - 대중적이고 강력한 사운드, 다양한 라인업과 합리적 가격대가 특징.
  - 스포츠용, 휴대용, 파티용 스피커에 강점.
  - 활동적이고 다이나믹한 사운드를 선호하는 사용자에게 인기.

- **하만카돈 (Harman/Kardon)**
  - 프리미엄 오디오 브랜드로 고급스럽고 세련된 디자인과 밸런스 좋은 음질로 유명.
  - 홈 오디오 및 하이엔드 오디오 시스템에 주력.
  - 클래식하면서도 현대적인 디자인과 음질을 중시하는 사용자에게 적합.

---

## 2. 음질 비교

| 구분          | JBL                                       | 하만카돈 (Harman/Kardon)                    |
|---------------|-------------------------------------------|--------------------------------------------|
| 음향 특성     | 강력한 베이스와 선명한 고음, 파워풀한 출력 | 밸런스가 뛰어나고 자연스러운 중음과 깨끗한 고음 |
| 사운드 튜닝   | 역동적이고 에너지 넘치는 사운드            | 고급스럽고 섬세한 음향, 음질의 정밀함 강조   |
| 주 용도       | 야외, 파티, 휴대용 스피커에 적합             | 홈시어터, 프리미엄 음악 감상용              |

---

## 3. 디자인 비교

- **JBL**
  - 견고하고 실용적인 디자인.
  - 방수 및 충격 방지 기능이 강화된 모델이 많아 야외 활동에 적합.
  - 다채로운 컬러와 젊고 활동적인 이미지 강조.

- **하만카돈**
  - 세련되고 미니멀한 디자인, 프리미엄 소재 사용.
  - 고급스러운 알루미늄 바디와 유려한 곡선 디자인.
  - 인테리어 소품으로도 훌륭한 미적 감각 제공.

---

## 4. 기술적 차별점

| 구분              | JBL                                         | 하만카돈 (Harman/Kardon)                   |
|-------------------|---------------------------------------------|--------------------------------------------|
| 무선 연결 기술    | Bluetooth, PartyBoost(다중 스피커 연결) 지원  | Bluetooth, Wi-Fi 지원 및 멀티룸 오디오 기능 제공 |
| 음향 보정 기술    | JBL 사운드 시스템 자체 튜닝 기술 적용          | 하만카돈 특유의 고음질 DSP(디지털 신호 처리) 탑재 |
| 배터리 및 내구성  | 야외 사용에 최적화된 긴 배터리 수명 및 내구성    | 주로 실내 사용 중심, 디자인과 음질 우선           |
| 스마트 기능      | 음성 비서 지원(구글 어시스턴트, Alexa 등)       | 고급 스마트 홈 오디오 시스템과 연동 가능          |

---

## 5. 요약 및 추천

| 기준           | JBL                                           | 하만카돈 (Harman/Kardon)                      |
|----------------|------------------------------------------------|------------------------------------------------|
| 추천 대상      | 활동적이고 파워풀한 사운드를 원하는 사용자, 야외 활동용 | 고품질 음향과 세련된 디자인, 프리미엄 홈오디오를 선호하는 사용자 |
| 가격대        | 비교적 합리적, 가성비 좋은 모델 많음             | 고가의 프리미엄 제품군 중심                        |
| 활용도        | 휴대성, 내구성 우수                              | 인테리어 및 홈시어터용                               |

---

### 결론

- **JBL**은 강력하고 에너지 넘치는 사운드, 야외에서의 뛰어난 내구성과 휴대성으로 활동적인 라이프스타일에 적합합니다.
- **하만카돈**은 고급스럽고 정교한 음질과 디자인으로 프리미엄 홈 오디오 시장에 적합하며, 섬세하고 밸런스 좋은 음향을 제공합니다.

삼성전자는 두 브랜드 모두를 통해 다양한 사용자 니즈를 충족시키고 있으며, 개인의 용도와 취향에 따라 최적의 선택이 가능하도록 다양한 모델을 판매 중입니다.

필요하신 경우 구체적인 모델별 성능이나 가격 정보도 추가로 안내해 드릴 수 있습니다!

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데, 새 이어버드를 기존 버즈와 함께 페어링하는 방법을 자세히 알려주세요. 한쪽 이어버드 페어링 절차와 주의사항도 포함해 설명해 주세요.
Google Search Results: 5


안녕하세요! 갤럭시 버즈 이어버드 한쪽을 새로 구매하셨을 때 기존 버즈와 함께 페어링하는 방법에 대해 자세히 안내드리겠습니다.

---

## 갤럭시 버즈 이어버드 한쪽 새로 구입 후 페어링 방법

### 1. 한쪽 이어버드 충전 케이스에 넣기
- 새로 구입한 이어버드와 기존 이어버드를 충전 케이스의 좌우 슬롯 홈에 맞게 각각 한쪽씩 넣습니다.
- 이어버드가 충전 단자에 정상적으로 안착되어야 하며, 충전 케이스 LED가 녹색 또는 빨간색으로 깜빡이는지 확인하세요.

### 2. 충전 케이스 커플링 모드 진입
- 충전 케이스 뚜껑을 열어둔 상태에서, 케이스 아래 충전 단자 옆에 있는 연결 버튼을 7초 이상 길게 누릅니다.
- 이때 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면 커플링이 완료된 것입니다.

### 3. 모바일 기기와 블루투스 연결
- 이어버드를 케이스에서 꺼내 모바일 기기와 블루투스로 연결합니다.
- Galaxy Wearable(갤럭시 웨어러블) 앱을 통해서도 연결 상태 및 설정을 확인할 수 있습니다.

---

## 주의사항 및 참고사항

- **소프트웨어 버전 확인**  
  기존 이어버드와 새로 구입한 이어버드 양쪽의 소프트웨어 버전이 동일하지 않으면 페어링 모드에 진입하지 못할 수 있습니다. 이 경우, 모바일 기기에서 Galaxy Wearable 앱을 이용하여 이어버드 소프트웨어를 업데이트한 후 다시 시도하세요.

- **배터리 타입 일치 여부 확인**  
  양쪽 이어버드의 배터리 타입이 다를 경우 커플링 작업이 불가능합니다. 예를 들어, 2019년 9월 생산분 이어버드를 분실하고 2019년 10월 이후 출시된 이어버드를 구매하면 배터리 타입이 다를 수 있으니 구매 전 꼭 확인해 주세요.

- **커플링 실패 시 조치**  
  - 이어버드 배터리 상태 표시등이 빨간색으로 계속 깜빡이는 경우, 이어버드를 충전 케이스에서 꺼냈다가 다시 넣고 7초 이상 길게 터치하여 커플링을 다시 시도하세요.
  - 충전 케이스 단자 접점 부위에 이물질이나 부식, 침수 여부를 확인해 주시기 바랍니다.

- **서비스센터 방문 안내**  
  모델에 따라서는 고객님이 직접 커플링 설정이 어려울 수 있으며, 이 경우 삼성 서비스센터 방문 후 엔지니어를 통해 커플링 작업 및 이어버드 구입 처리를 받으셔야 합니다.

---

필요시 Galaxy Wearable 앱을 통해 소프트웨어 업데이트 및 설정도 함께 진행해 주시면 원활한 사용이 가능합니다.  
추가 문의 있으시면 언제든지 도움 드리겠습니다. 감사합니다!

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 S24 차이
LLM Query: 삼성전자 S25 모델의 무게가 이전 모델인 S24와 비교했을 때 얼마나 차이가 나는지 상세하게 알려 주세요. 두 모델의 무게 차이와 관련된 정보를 중심으로 설명해 주세요.
Google Search Results: 5


삼성전자 갤럭시 S25 모델과 이전 모델인 갤럭시 S24 모델의 무게 차이에 대해 상세히 설명드리겠습니다.

---

## 갤럭시 S25 모델 무게 vs 갤럭시 S24 모델 무게 비교

| 모델          | 무게 (g) | 무게 변화 및 특징                            |
|--------------|----------|--------------------------------------------|
| 갤럭시 S24    | 167g     | 기존 모델, 두께 7.6mm                        |
| 갤럭시 S25    | 162g     | 5g 가벼워짐, 두께 7.2mm로 더 얇아짐         |

- **무게 차이:** 갤럭시 S25는 갤럭시 S24 대비 약 5g 가벼워졌습니다.
- **두께 변화:** S24의 7.6mm 두께에서 S25는 7.2mm로 더 얇아져 무게 경량화에 기여했습니다.
- **디자인:** S25는 강화된 아머 알루미늄 프레임을 적용하면서도 무게를 줄이는 데 성공했습니다.

---

### 추가 참고사항

- 갤럭시 S25 시리즈는 기본형 외에도 플러스, 울트라 모델이 있으며, 각각 무게가 다릅니다.
- S25 플러스 모델은 190g, 울트라 모델은 218g으로, 각각 S24 시리즈 대비 약간의 경량화가 이루어졌습니다.
- 울트라 모델은 특히 이전보다 그립감과 디자인이 개선되어 둥근 모서리와 무게 감소(232g → 218g)가 특징입니다.

---

## 요약

- **갤럭시 S25 기본형**은 **갤럭시 S24 기본형 대비 5g 가벼워진 162g**입니다.
- 두께도 더 얇아져 휴대성과 사용성이 개선되었습니다.
- 플러스 및 울트라 모델 역시 전작보다 경량화되었으며, 특히 울트라는 14g 정도 무게가 줄어들어 그립감이 좋아졌습니다.

---

더 자세한 스펙 및 비교는 삼성전자 공식 홈페이지 또는 삼성닷컴에서 확인하실 수 있습니다. 필요하시면 추가 정보도 제공해드리겠습니다!